# Lecture 4 #
## Monte Carlo Methods ##

Oftentimes, we can get approximate solutions to complicated problems by using randomness in our algorithms. The algorithms that give approximate solutions are called Monte Carlo Algorithms. (This is in contrast to the less common Las Vegas Algorithms, which use randomness and are guaranteed to give an exact solution).

Below, we will approximate the value of $\pi$ by randomly generating points $(x,y)$ in the first quadrant such that $x,y\in[0,1]$ and checking if each point lie inside of outside of the unit circle. We know the ratio of points in the unit circle to the total number of points generated should be approximately the area of a quarter of the unit circle, thus:

$$ \frac{\pi}{4} \approx \frac{\text{number of points inside the unit circle}}{\text{total number of points tested}} $$

Thus 

$$ \pi \approx 4*\frac{\text{number of points inside the unit circle}}{\text{total number of points tested}} $$

In [23]:
import random as random

#This is the number of points to check
numpoints = 10000
#We will check which points lie inside of the unit circle
count = 0
for i in range(numpoints):
    #This checks if the randomly generated point is inside the circle
    if sum([random.random()**2 for j in range(2)]) <= 1:
        count += 1

print(4*count/numpoints)

3.1196


The approximation isn't great, but we can still use this method for other problems.

## Exercise 1 ##
Find the approximate volume of the unit sphere using a Monte-Carlo Method.

In [17]:
numpoints = 1000000
count = 0
for i in range(numpoints):
    #This checks if the randomly generated point is inside the sphere
    if sum([random.random()**2 for j in range(3)]) <= 1:
        count += 1

#Here we have to multiply by 8 instead, because we are only checking in the first octant.
print(8*count/numpoints)
#This is the actual value
print(4/3*3.14159)

4.189072
4.188786666666666


## Central Limit Theorem ##
Central Limit Theorem can help us get more accurate approximations. Instead of checking the count for the total number of points, break up numpoints into smaller samples. For each of these samples, find the approximation for $\pi$ and then take the average of these approximations.

In [38]:
import random as random

numpoints = 10000
counts = []
#Here we will find the counts for each of the 10 samples of 100000 points
for j in range(10):
    count = 0
    for i in range(numpoints//10):
        #This checks if the randomly generated point is inside the circle
        if sum([random.random()**2 for j in range(2)]) <= 1:
            count += 1
    counts.append(count)

#These are the approximations for each of the samples
approximations = [4*count/(numpoints//10) for count in counts]
#This is the mean of the approximations
mean = sum(approximations)/10
print(mean)

3.1304


We get better accuracy in our approximation! Linearity of expectation tells us that we will get the same mean, but central limit theorem will lower the variance in the mean and also becomes helpful when running code in parallel. Each core can compute a sample mean for a separate sample, and this allows for better Monte Carlo simulations. 

## Optimization ##
Monte Carlo methods can also be used to get approximations for optimal solutions when doing optimization problems. Suppose we wish to optimize the function

$$ f(x) = 3x^3 + 2x^2 + 4 $$ 

for $x\in[-10,10] $. Using calculus, we can find the local extrema and test the endpoints. Alternatively, we can randomly sample some points, and see which gives the maximum and minimum values.

In [4]:
import random as random

#The optimal values
maximum = 0
minimum = 0
#The x values corresponding to these optimal values
maxx = 0
minx = 0

for i in range(100000):
    
    #Randomly pick x, and check the value of fx
    x = random.uniform(-10,10)
    fx = 3*x**3 + 2*x**2 + 4
    
    if fx < minimum:
        minimum = fx
        minx = x
    elif fx> maximum:
        maximum = fx
        maxx = x
        
print(maxx)
print(minx)

9.99945112522989
-9.999606502617464


This gives us values very close to the maximum and minimum x values.

## Exercise 2 ##
Using Monte Carlo methods, find the approximate maximum and minimum of 
$$ f(x) =\frac {(x \text{mod}3* x \text{mod}1 )} {x^2 + 1} $$

for $x \in [-10,10]$

In [5]:
def f(x):
    '''This is the function we want to optimize '''
    return (x%3 * x%1) / (x**2 + 1)

maximum = 0
maxx = 0
minimum = 0
minx = 0

#This part is similar to the above cell

for i in range(100000):
    
    x = random.uniform(-10,10)
    fx = f(x)
    
    if fx < minimum:
        minimum = fx
        minx = x
    elif fx> maximum:
        maximum = fx
        maxx = x
        
print(maxx)
print(minx)

-0.00015498769895216924
0


As our function is not continuous, we need to be careful. There isn't a maximum value as extreme value theorem doesn't apply. We can still get a large value by using Monte Carlo simulations, and if we test the output, we can still learn a lot about the given function.
## Approximate Probabilities ##
Suppose we have a coin with a $0.25$ probability of landing heads, and a $0.75$ probability of landing tails. We wish to find how many flips on average it will take for us to get $2$ heads. This can be simulated using Monte Carlo simulations (Note, this is a sum of geometric distributions, if you wish to use statistics instead).

In [6]:
#We could set up our probabilities here, if we do not wish to use a weights list
flips = ['h','t','t','t']

#These are the number of times we need to flip a coin until heads occurs twice
lengths = []
numtrials = 10000

for i in range(numtrials):
    
    #Reset the length and count at the start of each loop
    length = 0
    count = 0
    
    #This will keep the while loop running until we have heads twice. We shoul
    while count < 2:
        #If we flip heads
        if random.choice(flips) == 'h':
            length += 1
            count += 1
        #Otherwise
        else:
            length += 1
            
    #Add the lenght to lengths after we flip heads twice
    lengths.append(length)
    
print(sum(lengths)/numtrials)

8.0511


## Exercise 3 ##
A drunken random walk is a Markov Process where the state changes uniformly at random from the current state to an adjacent state. For example, on the number line, at each time step, the walker may change position to the next integer higher or the next integer lower than the current state. Consider this walk, where $X_n$ is the position of the walker at time step $n$. Then 

$$ P(X_{n+1}|X_n) = P(X_{n-1}|X_n) = \frac{1}{2} $$

In $100$ time steps, what is the expected maximum position reached by the random walker?

In [11]:
#We will track the max position from each trial
maxpositions = []

numtrials = 10000
numsteps = 100
#We either increase or decrease the position by 1
choices = [1,-1]

for i in range(numtrials):
    
    for j in range(numsteps):
        currentpos = 0
        maxpos = 0
        
        #Increase position by 1
        if random.choice(choices) == 1:
            currentpos += 1
            
            #Check the maximum
            if currentpos > maxpos:
                maxpos += 1
        
        #Decrease position by 1
        else:
            currentpos -= 1
    
    maxpositions.append(maxpos)

#Find the expected maxposition
print(sum(maxpositions)/numtrials)

0.4955
